In [7]:

import numpy as np 
import cv2
import os
import matplotlib.pyplot as plt

print(os.getcwd())
directory = os.path.join('/' + os.path.join(*os.getcwd().split('/')[0:-1]), "Output/calibration")


# Get the file paths of all the images in the directory
image_paths = [os.path.join(directory, "chess"+str(i+1)+".jpg") for i in range(5)]

# Read each image
images = []
for image_path in image_paths:
    image = cv2.imread(image_path)
    if image is not None:
        images.append(image)

images = images[::2]

# convert images to 1920x1080 format
for i in range(len(images)):
    # first crop to 3840x2160
    images[i] = images[i][340:2500, 216:4056]

    images[i] = cv2.resize(images[i], (1920, 1080))
    # save image
    cv2.imwrite(os.path.join(directory, "chess_smol"+str(i+1)+".jpg"), images[i])

# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 

checkerboard_size = (6,7)   # Number of inner corners per a chessboard row/column, which will be use in cv2.findChessboardCorners()
checkerboard_length = 50    # the board grid size you choose

objp = np.zeros((1, checkerboard_size[0] * checkerboard_size[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:checkerboard_size[0], 0:checkerboard_size[1]].T.reshape(-1, 2)
prev_img_shape = None


for img in images:
    ret, corners = cv2.findChessboardCorners(img, checkerboard_size, flags = (cv2.CALIB_USE_INTRINSIC_GUESS + cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE)) # find the corners
    print(ret)

    if ret:
        objpoints.append(objp)
        criteria=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 3, 0.1) # stop after 3 iterations or the accuracy of 0.1 is achieved
        img_grey=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        corners2=cv2.cornerSubPix(img_grey,corners,(5,5),(-1,-1),criteria) 
        imgpoints.append(corners2)
        
    img_corner=cv2.drawChessboardCorners(img,checkerboard_size,corners2,ret) # draw detected corners
    img_corner2 = cv2.cvtColor(img_corner,cv2.COLOR_BGR2RGB)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_grey.shape[::-1], None, None)




/Users/alistair/Projects/Dissertation/Jenson/src/Utils
True
False
True


In [8]:
print(mtx)

[[1.76547132e+03 0.00000000e+00 9.13837132e+02]
 [0.00000000e+00 1.75659718e+03 6.09839517e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [9]:
print(dist)

[[-7.87585530e-01  6.44333466e+00 -1.48158605e-03  1.70506714e-02
  -2.85195362e+01]]
